In [1]:
pip install -q torch transformers langchain_chroma bitsandbytes langchain faiss-gpu langchain_huggingface langchain-community sentence-transformers  pacmap tqdm matplotlib datasets

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
pip install -q sentence_transformers

In [3]:
!python --version

Python 3.12.12


In [4]:
from tqdm.notebook import tqdm
import pandas as pd
import os
import csv
import sys
import numpy as np
import time
import random
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
import textwrap
import torch

import random
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import accuracy_score

## Processing dataset

In [5]:
import json
from huggingface_hub import hf_hub_download

filepath = hf_hub_download(
    repo_id='McAuley-Lab/Amazon-C4',
    filename='sampled_item_metadata_1M.jsonl',
    repo_type='dataset'
)

item_pool = []
with open(filepath, 'r') as file:
    for line in file:
        item_pool.append(json.loads(line.strip()))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sampled_item_metadata_1M.jsonl:   0%|          | 0.00/643M [00:00<?, ?B/s]

In [6]:
from datasets import load_dataset

dataset = load_dataset('McAuley-Lab/Amazon-C4')['test']

README.md: 0.00B [00:00, ?B/s]

test.csv:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/21223 [00:00<?, ? examples/s]

In [7]:
item_metadata_map = {item['item_id']: {'metadata': item['metadata'], 'category': item['category']} for item in item_pool}

In [8]:
# new_list = []
# for data in dataset:
#     item_id = data['item_id']
#     item_info = item_metadata_map.get(item_id, {'metadata': None, 'category': None})  # 找到 metadata 和 category
#     new_entry = {
#         'query': data['query'],
#         'item_id': item_id,
#         'metadata': item_info['metadata'],
#         'category': item_info['category']
#     }
#     new_list.append(new_entry)

In [9]:
new_list = []
for data in dataset:
    item_id = data['item_id']
    item_info = item_metadata_map.get(item_id, {'metadata': None, 'category': None})

    metadata = item_info['metadata']
    if metadata is None or len(metadata.split()) < 10:
        continue

    new_entry = {
        'query': data['query'],
        'item_id': item_id,
        'metadata': metadata,
        'category': item_info['category']
    }
    new_list.append(new_entry)


In [10]:
print(len(new_list))

20250


In [11]:
queries = []
passages = []


limit = max(1, len(new_list) // 20)

for idx, entry in enumerate(new_list):
    if idx < limit:
        queries.append(f"query: {entry['query']}")
        passages.append(f"passage: {entry['metadata']}")

input_texts = queries + passages

# print(input_texts)


In [12]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

random.shuffle(new_list)
split_idx = int(0.9 * len(new_list))
train_pool = new_list[:split_idx]
test_pool = new_list[split_idx:]

def prepare_dataset(pool):
    # 修改：使用 query 而不是 metadata 来训练分类器
    queries = [item['query'] for item in pool if item['query'] and item['category']]
    categories = [item['category'] for item in pool if item['query'] and item['category']]
    return queries, categories

train_queries, train_categories = prepare_dataset(train_pool)
test_queries, test_categories = prepare_dataset(test_pool)

category_to_idx = {category: idx for idx, category in enumerate(set(train_categories))}
idx_to_category = {idx: category for category, idx in category_to_idx.items()}
train_labels = [category_to_idx[cat] for cat in train_categories]
test_labels = [category_to_idx[cat] for cat in test_categories]

print(f"Training samples: {len(train_queries)}")
print(f"Test samples: {len(test_queries)}")
print(f"Number of categories: {len(category_to_idx)}")

Training samples: 18225
Test samples: 2025
Number of categories: 30


In [13]:
class QueryDataset(Dataset):
    def __init__(self, queries, labels, tokenizer, max_len=128):
        self.queries = queries
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, idx):
        query = self.queries[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            query,
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

train_dataset = QueryDataset(train_queries, train_labels, tokenizer)
test_dataset = QueryDataset(test_queries, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [15]:
class QueryClassifier(nn.Module):
    def __init__(self, num_categories):
        super(QueryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-large-uncased")
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_categories)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_token = outputs.last_hidden_state[:, 0, :]  # [CLS] token embedding
        logits = self.classifier(cls_token)
        return logits

num_categories = len(category_to_idx)
model = QueryClassifier(num_categories).to(device)

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [16]:
def train_model(model, train_loader, test_loader, num_epochs=3, lr=1e-5):
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    return model

model = train_model(model, train_loader, test_loader, num_epochs=4)
classifier_model = model

Training Epoch 1: 100%|██████████| 570/570 [04:49<00:00,  1.97it/s]


Epoch 1, Loss: 1.7836


Training Epoch 2: 100%|██████████| 570/570 [04:48<00:00,  1.97it/s]


Epoch 2, Loss: 1.0780


Training Epoch 3: 100%|██████████| 570/570 [04:48<00:00,  1.97it/s]


Epoch 3, Loss: 0.8272


Training Epoch 4: 100%|██████████| 570/570 [04:48<00:00,  1.97it/s]

Epoch 4, Loss: 0.6277


In [17]:
def evaluate_model(model, test_loader, top_k=3):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask)
            _, top_preds = torch.topk(outputs, k=top_k, dim=-1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(top_preds.cpu().numpy())

    top_k_accuracy = 0
    for label, preds in zip(all_labels, all_predictions):
        if label in preds:
            top_k_accuracy += 1

    top_k_accuracy /= len(all_labels)
    print(f"Top-{top_k} Accuracy: {top_k_accuracy:.4f}")

evaluate_model(classifier_model, test_loader, top_k=4)


Evaluating: 100%|██████████| 64/64 [00:12<00:00,  5.27it/s]

Top-4 Accuracy: 0.9022


In [19]:
from torch.nn.functional import softmax

result_list = []

model.eval()

print("Running inference: Using trained query classifier to predict categories...")

with torch.no_grad():
    for batch_idx, batch in enumerate(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # 现在 test_loader 中是真实的 query，直接用来预测类别
        logits = model(input_ids, attention_mask)
        probabilities = softmax(logits, dim=-1)

        # 获取 top-2 预测类别 (categories # changed to 4)
        top2_indices = torch.topk(probabilities, 4, dim=-1).indices.cpu().numpy()

        for i, top2 in enumerate(top2_indices):
            top2_categories = [idx_to_category[idx] for idx in top2]

            # 根据预测的类别筛选候选商品
            matched_items = [
                {
                    "item_id": item["item_id"],
                    "metadata": item["metadata"],
                    "category": item["category"],
                }
                for item in new_list
                if item["category"] in top2_categories
            ]

            # 获取对应的真实 query 和 ground truth
            test_idx = batch_idx * test_loader.batch_size + i
            result_list.append({
                "query": test_pool[test_idx]['query'],
                "real_category": test_pool[test_idx]['category'],
                "real_item_id": test_pool[test_idx]['item_id'],
                "top2_categories": top2_categories,
                "matched_items": matched_items
            })

print(f"✓ Processed {len(result_list)} test queries")
print(f"  Average candidate pool size: {sum(len(r['matched_items']) for r in result_list) / len(result_list):.0f}")

Running inference: Using trained query classifier to predict categories...


KeyboardInterrupt: 

## ✅ 改进说明：Query-Based 分类器

**主要修改**:
- **Cell 12**: `prepare_dataset()` 现在使用 `item['query']` 而不是 `item['metadata']`
- **Cell 18**: 推理时直接用 query 预测类别，无需额外添加

**新的训练流程**:
```
用户 query → BERT QueryClassifier → Top-2 categories → 候选商品池
```

**优势**:
1. ✅ Query 真正参与训练
2. ✅ 推理流程符合实际应用 (用户只提供 query)
3. ✅ 代码更简洁，逻辑更清晰

**下一步**: 重新运行 Cell 12-18 来训练新的 query 分类器

In [20]:
# Save the trained QueryClassifier model
torch.save(model.state_dict(), 'query_classifier.pth')
classifier_model = model  # Keep a reference before it gets overwritten
print("✓ Saved query_classifier.pth")
print(f"  Model has {num_categories} output categories")

✓ Saved query_classifier.pth
  Model has 30 output categories


In [21]:
# -*- coding: utf-8 -*-

import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import gc

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base').to(device)

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [22]:
for result in result_list:
    query = []
    passages = []
    query.append(f"query: {result['query']}")
    for matched_item in result['matched_items']:
        passages.append(f"passage: {matched_item['metadata']}")
    # print(query)
    input_texts = query + passages
    batch_dict = tokenizer(
        input_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

    batch_dict = {key: value.to(device) for key, value in batch_dict.items()}

    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

    embeddings = F.normalize(embeddings, p=2, dim=1)

    scores = (embeddings[:1] @ embeddings[1:].T) * 100
    print(scores)
    break


tensor([[73.9798, 75.9833, 77.4119,  ..., 79.6367, 78.5389, 77.7805]],
       device='cuda:0')


In [23]:
import torch
import torch.nn.functional as F

for result in result_list:
    query = []
    passages = []
    query.append(f"query: {result['query']}")

    for matched_item in result['matched_items']:
        passages.append(f"passage: {matched_item['metadata']}")

    # Combine query and passages into input_texts
    input_texts = query + passages

    # Tokenize inputs
    batch_dict = tokenizer(
        input_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    batch_dict = {key: value.to(device) for key, value in batch_dict.items()}

    # Get embeddings
    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)

    # Compute scores
    scores = (embeddings[:1] @ embeddings[1:].T) * 100  # Compute similarity scores
    scores = scores.squeeze(0)  # Remove extra dimension for easier processing

    # Get top-10 scores and corresponding indices
    top_scores, top_indices = torch.topk(scores, k=1000)

    # Map top scores to matched_items and check their IDs
    top_matched_items = [result['matched_items'][idx] for idx in top_indices]
    print(result['real_item_id'])
    for i, matched_item in enumerate(top_matched_items):
        item_id = matched_item['item_id']
        is_real_item = item_id == result['real_item_id']
        print(f"Rank {i+1}: Score = {top_scores[i].item():.2f}, Item ID = {item_id}, Is Real Item: {is_real_item}")

    break  # Break after processing the first result (for debugging)


B0BP6WWSBD
Rank 1: Score = 84.28, Item ID = B09D2TRSHM, Is Real Item: False
Rank 2: Score = 83.96, Item ID = B0B6BD13Q9, Is Real Item: False
Rank 3: Score = 83.87, Item ID = B0BP6WWSBD, Is Real Item: True
Rank 4: Score = 83.87, Item ID = B0BP6WWSBD, Is Real Item: True
Rank 5: Score = 83.67, Item ID = B0B464RB6B, Is Real Item: False
Rank 6: Score = 83.42, Item ID = B0C8V52BLR, Is Real Item: False
Rank 7: Score = 83.36, Item ID = B0C5H87577, Is Real Item: False
Rank 8: Score = 83.31, Item ID = B0B451WX1G, Is Real Item: False
Rank 9: Score = 83.30, Item ID = B09F35NTYT, Is Real Item: False
Rank 10: Score = 83.30, Item ID = B09F35NTYT, Is Real Item: False
Rank 11: Score = 83.04, Item ID = B0BYTJ6LDR, Is Real Item: False
Rank 12: Score = 83.02, Item ID = B0923LNLK7, Is Real Item: False
Rank 13: Score = 82.97, Item ID = B07JLTMQJT, Is Real Item: False
Rank 14: Score = 82.81, Item ID = B085C2HMQP, Is Real Item: False
Rank 15: Score = 82.76, Item ID = B08R2N5SDX, Is Real Item: False
Rank 16: S

In [24]:
print(len(result_list[2]['matched_items']))

4225


In [25]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

real_item_in_top200_count = 0
total_results = min(100, len(result_list))

for result in tqdm(result_list[:100], desc="Processing Results", unit="result"):
    query = []
    passages = []
    query.append(f"query: {result['query']}")

    for matched_item in result['matched_items']:
        passages.append(f"passage: {matched_item['metadata']}")

    # Combine query and passages into input_texts
    input_texts = query + passages

    # Tokenize inputs
    batch_dict = tokenizer(
        input_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    batch_dict = {key: value.to(device) for key, value in batch_dict.items()}

    # Get embeddings
    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)

    # Compute scores
    scores = (embeddings[:1] @ embeddings[1:].T) * 100  # Compute similarity scores
    scores = scores.squeeze(0)  # Remove extra dimension for easier processing

    # Get top-10 scores and corresponding indices
    top_scores, top_indices = torch.topk(scores, k=200)

    # Map top scores to matched_items and check if real_item is in top-10
    top_matched_items = [result['matched_items'][idx] for idx in top_indices]

    # Check if real_item is in top-10
    real_item_found = any(matched_item['item_id'] == result['real_item_id'] for matched_item in top_matched_items)

    # Update count if real_item is found in top-10
    if real_item_found:
        real_item_in_top200_count += 1

# Calculate the probability
if total_results > 0:
    probability = real_item_in_top200_count / total_results
else:
    probability = 0.0

print(f"\nProbability of real_item appearing in top-200: {probability:.2%}")


Processing Results: 100%|██████████| 100/100 [11:46<00:00,  7.06s/result]


Probability of real_item appearing in top-200: 78.00%


In [ ]:
# extend to all results
import torch
import torch.nn.functional as F
from tqdm import tqdm

real_item_in_top200_count = 0
total_results = len(result_list)

for result in tqdm(result_list, desc="Processing Results", unit="result"):
    query = []
    passages = []
    query.append(f"query: {result['query']}")

    for matched_item in result['matched_items']:
        passages.append(f"passage: {matched_item['metadata']}")

    # Combine query and passages into input_texts
    input_texts = query + passages

    # Tokenize inputs
    batch_dict = tokenizer(
        input_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    batch_dict = {key: value.to(device) for key, value in batch_dict.items()}

    # Get embeddings
    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)

    # Compute scores
    scores = (embeddings[:1] @ embeddings[1:].T) * 100  # Compute similarity scores
    scores = scores.squeeze(0)  # Remove extra dimension for easier processing

    # Dynamically determine top-k value
    k = min(200, scores.size(0))  # Ensure k does not exceed the number of scores

    # Get top-k scores and corresponding indices
    top_scores, top_indices = torch.topk(scores, k=k)

    # Map top scores to matched_items and check if real_item is in top-k
    top_matched_items = [result['matched_items'][idx] for idx in top_indices]

    # Check if real_item is in top-k
    real_item_found = any(matched_item['item_id'] == result['real_item_id'] for matched_item in top_matched_items)

    # Update count if real_item is found in top-k
    if real_item_found:
        real_item_in_top200_count += 1

# Calculate the probability
if total_results > 0:
    probability = real_item_in_top200_count / total_results
else:
    probability = 0.0

print(f"\nProbability of real_item appearing in top-200: {probability:.2%}")


In [ ]:
import torch
import pickle

# Save category mappings (required for inference)
mappings = {
    'category_to_idx': category_to_idx,
    'idx_to_category': idx_to_category,
    'num_categories': num_categories
}

with open('category_mappings.pkl', 'wb') as f:
    pickle.dump(mappings, f)

print("Saved category_mappings.pkl")
print(f"Total categories: {num_categories}")

# NOTE: The QueryClassifier model needs to be saved BEFORE cell 20
# because the 'model' variable gets overwritten with the E5 model.
#
# To save the QueryClassifier properly, add this cell between cell 18 and 20:
#
# # Save trained QueryClassifier
# torch.save(model.state_dict(), 'query_classifier.pth')
# classifier_model = model  # Keep a reference
# print("Saved query_classifier.pth")
#
# The E5 model doesn't need saving as it's loaded from HuggingFace

print("\n⚠️  WARNING: QueryClassifier model not saved!")
print("   Add the save code between cells 18-20 to preserve the trained model.")

## Loading Saved Weights for Inference

To reuse the trained model in a new session, use the following code:

In [ ]:
# Example: How to load saved weights for inference
import torch
import pickle
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel
from torch import nn

# 1. Load category mappings
with open('category_mappings.pkl', 'rb') as f:
    mappings = pickle.load(f)

category_to_idx = mappings['category_to_idx']
idx_to_category = mappings['idx_to_category']
num_categories = mappings['num_categories']

print(f"Loaded {num_categories} categories")

# 2. Reconstruct QueryClassifier model architecture
class QueryClassifier(nn.Module):
    def __init__(self, num_categories):
        super(QueryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-large-uncased")
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_categories)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_token = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_token)
        return logits

# 3. Load trained weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier = QueryClassifier(num_categories).to(device)
classifier.load_state_dict(torch.load('query_classifier.pth', map_location=device))
classifier.eval()

print("✓ QueryClassifier loaded successfully")

# 4. Load E5 model for semantic similarity
e5_tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
e5_model = AutoModel.from_pretrained('intfloat/multilingual-e5-base').to(device)

print("✓ E5 model loaded successfully")
print("\nModels ready for inference!")

## Optimized Version - Pre-compute Embeddings

The original approach recomputes embeddings for the same items repeatedly. This optimized version pre-computes all item embeddings once, reducing runtime from ~2 hours to ~5-10 minutes.

In [26]:
# Step 1: Pre-compute embeddings for all unique items
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Create a mapping of item_id to metadata
item_id_to_metadata = {item['item_id']: item['metadata'] for item in new_list}
unique_item_ids = list(item_id_to_metadata.keys())

print(f"Pre-computing embeddings for {len(unique_item_ids)} unique items...")

# Pre-compute embeddings in batches
item_embeddings = {}
batch_size = 128  # Adjust based on GPU memory

for i in tqdm(range(0, len(unique_item_ids), batch_size), desc="Computing item embeddings"):
    batch_ids = unique_item_ids[i:i+batch_size]
    batch_texts = [f"passage: {item_id_to_metadata[item_id]}" for item_id in batch_ids]

    # Tokenize batch
    batch_dict = tokenizer(
        batch_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    batch_dict = {key: value.to(device) for key, value in batch_dict.items()}

    # Get embeddings
    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embeddings = F.normalize(embeddings, p=2, dim=1)

    # Store embeddings
    for item_id, emb in zip(batch_ids, embeddings):
        item_embeddings[item_id] = emb.cpu()  # Move to CPU to save GPU memory

print(f"✓ Pre-computed {len(item_embeddings)} item embeddings")

Pre-computing embeddings for 19522 unique items...


Computing item embeddings: 100%|██████████| 153/153 [00:30<00:00,  4.94it/s]

✓ Pre-computed 19522 item embeddings


In [27]:
# Step 2: Compute query embeddings and evaluate (MUCH FASTER)
real_item_in_top200_count = 0
total_results = len(result_list)

# Process queries in batches for even more speedup
query_batch_size = 32

for batch_start in tqdm(range(0, len(result_list), query_batch_size), desc="Processing query batches"):
    batch_end = min(batch_start + query_batch_size, len(result_list))
    batch_results = result_list[batch_start:batch_end]

    # Prepare batch queries
    batch_query_texts = [f"query: {result['query']}" for result in batch_results]

    # Tokenize query batch
    query_batch_dict = tokenizer(
        batch_query_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    query_batch_dict = {key: value.to(device) for key, value in query_batch_dict.items()}

    # Get query embeddings
    with torch.no_grad():
        outputs = model(**query_batch_dict)
        query_embeddings = average_pool(outputs.last_hidden_state, query_batch_dict['attention_mask'])
        query_embeddings = F.normalize(query_embeddings, p=2, dim=1)

    # Process each query in the batch
    for idx, (result, query_emb) in enumerate(zip(batch_results, query_embeddings)):
        # Get pre-computed embeddings for matched items
        matched_item_ids = [item['item_id'] for item in result['matched_items']]

        # Stack embeddings for matched items
        matched_embeddings = torch.stack([
            item_embeddings[item_id].to(device) for item_id in matched_item_ids
        ])

        # Compute similarity scores
        scores = (query_emb.unsqueeze(0) @ matched_embeddings.T) * 100
        scores = scores.squeeze(0)

        # Get top-k
        k = min(200, scores.size(0))
        top_scores, top_indices = torch.topk(scores, k=k)

        # Check if real item is in top-k
        top_matched_items = [result['matched_items'][idx] for idx in top_indices.cpu().numpy()]
        real_item_found = any(item['item_id'] == result['real_item_id'] for item in top_matched_items)

        if real_item_found:
            real_item_in_top200_count += 1

# Calculate probability
probability = real_item_in_top200_count / total_results if total_results > 0 else 0.0
print(f"\nProbability of real_item appearing in top-200: {probability:.2%}")
print(f"Found {real_item_in_top200_count} out of {total_results} items")

Processing query batches: 100%|██████████| 4/4 [00:12<00:00,  3.15s/it]


Probability of real_item appearing in top-200: 77.88%
Found 88 out of 113 items


In [ ]:
# Optional: Save pre-computed embeddings for future use
torch.save(item_embeddings, 'item_embeddings.pth')
print(f"✓ Saved item embeddings to item_embeddings.pth")
print(f"  File contains {len(item_embeddings)} item embeddings")

# To load later:
# item_embeddings = torch.load('item_embeddings.pth')

## Additional Optimization Options

If you need even more speed:

1. **Use FAISS for similarity search** (2-5x faster for large candidate sets)
   - Pre-build FAISS index for all items
   - GPU-accelerated approximate nearest neighbor search
   
2. **Reduce candidate pool size**
   - Use top-1 category instead of top-2 (fewer candidates)
   - Pre-filter items before computing similarities
   
3. **Mixed precision (FP16)**
   - Use `torch.cuda.amp.autocast()` for faster inference
   - Can be 2x faster on modern GPUs

4. **Batch all queries at once** (if GPU memory allows)
   - Process all 2,025 queries in larger batches
   - Trade memory for speed

## Debugging Accuracy Differences

The batched version may produce slightly different results due to padding differences. Let's verify:

In [ ]:
# Compare embeddings from both methods for the first query
test_result = result_list[0]
test_query = f"query: {test_result['query']}"
test_items = [f"passage: {item['metadata']}" for item in test_result['matched_items'][:5]]

# Method 1: Original (tokenize together)
input_texts_together = [test_query] + test_items
batch_together = tokenizer(input_texts_together, max_length=128, padding=True, truncation=True, return_tensors='pt')
print(f"Original method - input_ids shape: {batch_together['input_ids'].shape}")
print(f"Padding length: {batch_together['input_ids'].shape[1]}")

with torch.no_grad():
    outputs = model(**{k: v.to(device) for k, v in batch_together.items()})
    emb_together = average_pool(outputs.last_hidden_state, batch_together['attention_mask'].to(device))
    emb_together = F.normalize(emb_together, p=2, dim=1)

# Method 2: Optimized (tokenize separately)
batch_query = tokenizer([test_query], max_length=128, padding=True, truncation=True, return_tensors='pt')
batch_items = tokenizer(test_items, max_length=128, padding=True, truncation=True, return_tensors='pt')

print(f"\nOptimized method - query shape: {batch_query['input_ids'].shape}")
print(f"Optimized method - items shape: {batch_items['input_ids'].shape}")

with torch.no_grad():
    q_out = model(**{k: v.to(device) for k, v in batch_query.items()})
    emb_query = average_pool(q_out.last_hidden_state, batch_query['attention_mask'].to(device))
    emb_query = F.normalize(emb_query, p=2, dim=1)

    i_out = model(**{k: v.to(device) for k, v in batch_items.items()})
    emb_items = average_pool(i_out.last_hidden_state, batch_items['attention_mask'].to(device))
    emb_items = F.normalize(emb_items, p=2, dim=1)

# Compare embeddings
print(f"\nQuery embedding difference: {torch.max(torch.abs(emb_together[0] - emb_query[0])).item():.6f}")
print(f"Item embeddings max difference: {torch.max(torch.abs(emb_together[1:] - emb_items)).item():.6f}")

# Compare scores
scores_original = (emb_together[:1] @ emb_together[1:].T) * 100
scores_optimized = (emb_query @ emb_items.T) * 100
print(f"\nScore difference: {torch.max(torch.abs(scores_original - scores_optimized)).item():.6f}")
print(f"Scores are identical: {torch.allclose(scores_original, scores_optimized, atol=1e-5)}")

## Fixed Version - Guaranteed Identical Results

The issue is `padding=True` uses different padding lengths per batch. Using `padding="max_length"` ensures identical results:

In [28]:
# Step 1: Pre-compute embeddings with FIXED padding
import torch
import torch.nn.functional as F
from tqdm import tqdm

item_id_to_metadata = {item['item_id']: item['metadata'] for item in new_list}
unique_item_ids = list(item_id_to_metadata.keys())

print(f"Pre-computing embeddings for {len(unique_item_ids)} unique items...")

item_embeddings_fixed = {}
batch_size = 128

for i in tqdm(range(0, len(unique_item_ids), batch_size), desc="Computing item embeddings"):
    batch_ids = unique_item_ids[i:i+batch_size]
    batch_texts = [f"passage: {item_id_to_metadata[item_id]}" for item_id in batch_ids]

    # Use padding="max_length" instead of padding=True
    batch_dict = tokenizer(
        batch_texts,
        max_length=128,
        padding="max_length",  # FIXED: Always pad to 128
        truncation=True,
        return_tensors='pt'
    )
    batch_dict = {key: value.to(device) for key, value in batch_dict.items()}

    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embeddings = F.normalize(embeddings, p=2, dim=1)

    for item_id, emb in zip(batch_ids, embeddings):
        item_embeddings_fixed[item_id] = emb.cpu()

print(f"✓ Pre-computed {len(item_embeddings_fixed)} item embeddings with fixed padding")

Pre-computing embeddings for 19522 unique items...


Computing item embeddings: 100%|██████████| 153/153 [00:30<00:00,  4.94it/s]

✓ Pre-computed 19522 item embeddings with fixed padding


In [29]:
# Step 2: Evaluate with FIXED padding (should match original exactly)
real_item_in_top200_count = 0
total_results = len(result_list)
query_batch_size = 32

for batch_start in tqdm(range(0, len(result_list), query_batch_size), desc="Processing query batches"):
    batch_end = min(batch_start + query_batch_size, len(result_list))
    batch_results = result_list[batch_start:batch_end]

    batch_query_texts = [f"query: {result['query']}" for result in batch_results]

    # Use padding="max_length" instead of padding=True
    query_batch_dict = tokenizer(
        batch_query_texts,
        max_length=128,
        padding="max_length",  # FIXED: Always pad to 128
        truncation=True,
        return_tensors='pt'
    )
    query_batch_dict = {key: value.to(device) for key, value in query_batch_dict.items()}

    with torch.no_grad():
        outputs = model(**query_batch_dict)
        query_embeddings = average_pool(outputs.last_hidden_state, query_batch_dict['attention_mask'])
        query_embeddings = F.normalize(query_embeddings, p=2, dim=1)

    for result, query_emb in zip(batch_results, query_embeddings):
        matched_item_ids = [item['item_id'] for item in result['matched_items']]

        matched_embeddings = torch.stack([
            item_embeddings_fixed[item_id].to(device) for item_id in matched_item_ids
        ])

        scores = (query_emb.unsqueeze(0) @ matched_embeddings.T) * 100
        scores = scores.squeeze(0)

        k = min(200, scores.size(0))
        top_scores, top_indices = torch.topk(scores, k=k)

        top_matched_items = [result['matched_items'][i] for i in top_indices.cpu().numpy()]
        real_item_found = any(item['item_id'] == result['real_item_id'] for item in top_matched_items)

        if real_item_found:
            real_item_in_top200_count += 1

probability = real_item_in_top200_count / total_results if total_results > 0 else 0.0
print(f"\nWith FIXED padding:")
print(f"Probability of real_item appearing in top-200: {probability:.2%}")
print(f"Found {real_item_in_top200_count} out of {total_results} items")

Processing query batches: 100%|██████████| 4/4 [00:12<00:00,  3.21s/it]


With FIXED padding:
Probability of real_item appearing in top-200: 77.88%
Found 88 out of 113 items
